# Task-1


In [2]:
import pandas as pd
import numpy as np

In [14]:

df = pd.read_csv("datasets\dataset-1.csv")
df.head()

,id_1,id_2,route,moto,car,rv,bus,truck
0,829,827,1,2.05,4.14,4.14,10.1,15.2
1,829,821,4,6.63,13.26,13.26,32.4,48.5
2,829,804,7,14.41,28.92,28.92,64.7,97.0
3,829,822,6,5.90,11.81,11.81,28.8,43.2
4,829,826,9,2.87,5.81,5.81,14.2,21.2


In [21]:
import pandas as pd

def get_bus_indexes(df) -> list:
    """
    Returns the indexes where the 'bus' values are greater than twice the mean.

    Args:
        df (pandas.DataFrame)

    Returns:
        list: List of indexes where 'bus' values exceed twice the mean.
    """
    # Calculate the mean of the 'bus' column
    mean_bus = df['bus'].mean()

    # Identify the indexes where 'bus' values are greater than twice the mean
    bus_indexes = df[df['bus'] > 2 * mean_bus].index.tolist()

    # Sort the indexes in ascending order
    bus_indexes.sort()

    return bus_indexes

# Example usage:
# Assuming 'df' is your DataFrame, you can call the function like this:
indices = get_bus_indexes(df)
print(indices)


[2, 7, 12, 17, 25, 30, 54, 64, 70, 97, 144, 145, 149, 154, 160, 201, 206, 210, 215, 234, 235, 245, 250, 309, 314, 319, 322, 323, 334, 340]
